In [2]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

In [3]:
dataset = fetch_california_housing()

In [4]:
data = pd.DataFrame(dataset.data,columns=dataset.feature_names)
data['Price'] = dataset.target

In [5]:
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


Train test split, Hyperparam Tuning, MLFlow Experements

In [6]:
from urllib.parse import urlparse
X = data.drop(columns=['Price'])
y = data['Price']

In [12]:
def HyperParameter_tuning(X_train,y_train,grid):
    rf = RandomForestRegressor()
    gridSearch = GridSearchCV(estimator=rf,param_grid=grid,cv=3,n_jobs=-1,verbose=2,
                                scoring = 'neg_mean_squared_error')
    gridSearch.fit(X_train,y_train)
    return gridSearch

In [13]:
# Split the data into train and test
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

from mlflow.models import infer_signature
signature = infer_signature(X_train,y_train)

In [14]:
#Define HyperParameter Grid

param_grid = {
    'n_estimators':[100,200],
    'max_depth':[None,10,5],
    'min_samples_split':[2,5],
    'min_samples_leaf':[1,2]
            }

            

In [16]:
#MLflow experement

with mlflow.start_run():

    # Create a new experement
    mlflow.set_experiment("Price Prediction")  
    #Hyper param tuning
    grid_search = HyperParameter_tuning(X_train,y_train,param_grid)

    #Get the best model
    best_model = grid_search.best_estimator_


    #Evalueate model
    predictions = best_model.predict(X_test)
    mse = mean_squared_error(y_test,predictions)

    #Log best params and metrics
    mlflow.log_param("best_n_estimators",grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth",grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split",grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf",grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mse",mse)

    mlflow.set_tracking_uri('http://127.0.0.1:5000')
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != 'file':
        mlflow.sklearn.log_model(best_model,"Model",registered_model_name='Best Randomforest Model',input_example = X_train)
    else:
        mlflow.sklearn.log_model(best_model,"Model",signature=set_tracking_uri)




Fitting 3 folds for each of 24 candidates, totalling 72 fits


Successfully registered model 'Best Randomforest Model'.
2025/01/09 12:38:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best Randomforest Model, version 1


🏃 View run trusting-bug-114 at: http://127.0.0.1:5000/#/experiments/271662853056853560/runs/29939e5c57bb43eebfc2c38249c8b41d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/271662853056853560


Created version '1' of model 'Best Randomforest Model'.
